In [195]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib

## Load data

In [196]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

In [197]:
# 550 data with 300 features
print(x_train.shape)

(550, 300)


In [198]:
# It's a binary classification problem 
print(np.unique(y_train))

[0 1]


## Question 1
K-fold data partition: Implement the K-fold cross-validation function. Your function should take K as an argument and return a list of lists (len(list) should equal to K), which contains K elements. Each element is a list contains two parts, the first part contains the index of all training folds, e.g. Fold 2 to Fold 5 in split 1. The second part contains the index of validation fold, e.g. Fold 1 in  split 1

In [199]:
def cross_validation(x_train, y_train, k=5):
    n_samples = x_train.shape[0]
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    folds = []
    size = n_samples//k + 1
    for i in range(n_samples % k):
        start = i * size
        fold = indices[start:start+size]
        folds.append(fold)
    size = n_samples // k
    for i in range(n_samples % k, k):
        start = i * size
        fold = indices[start:start+size]
        folds.append(fold)
    folds = np.asarray(folds)
    kfold = []
    for i in range(k):
        train = folds[np.arange(k) != i]
        val = folds[i]
        kfold.append([train.ravel(), val])
    return kfold

In [200]:
kfold_data = cross_validation(x_train, y_train, k=10)
assert len(kfold_data) == 10  # should contain 10 fold of data
assert len(kfold_data[0]) == 2  # each element should contain train fold and validation fold
assert kfold_data[0][1].shape[0] == 55  # The number of data in each validation fold should equal to training data divieded by K

## Question 2
Using sklearn.svm.SVC to train a classifier on the provided train set and conduct the grid search of “C” and “gamma” to find the best parameters by cross-validation.

In [201]:
def cal_mse(y_pred, y_test):
    return np.sum((y_pred-y_test)**2) / y_pred.shape[0]

In [202]:
def svm_gridsearch(x, y, kfold, cand_C, cand_gamma, is_regression=False):
    gridsearch = []
    max_acc, min_mse = 0, 1e10
    best_C, best_gamma = 0, 0
    for C in cand_C:
        t = []
        for gamma in cand_gamma:
            if is_regression:
                avg_mse = 0
                for f in kfold:
                    clf = SVR(C=C, kernel='rbf', gamma=gamma)
                    clf.fit(x[f[0]], y[f[0]])
                    y_pred = clf.predict(x[f[1]])
                    mse = cal_mse(y_pred, y[f[1]])
                    avg_mse += mse
                avg_mse /= len(kfold)
                print(f'C={C}, gamma={gamma}, avg mse={avg_mse:.2f}')
                t.append(avg_mse)
                if avg_mse <= min_mse:
                    best_C = C
                    best_gamma = gamma
                    min_mse = avg_mse
            else:
                avg_acc = 0
                for f in kfold:
                    clf = SVC(C=C, kernel='rbf', gamma=gamma)
                    clf.fit(x[f[0]], y[f[0]])
                    y_pred = clf.predict(x[f[1]])
                    acc = accuracy_score(y_pred, y[f[1]])
                    avg_acc += np.around(acc, 2)
                avg_acc /= len(kfold)
                print(f'C={C}, gamma={gamma}, avg acc={avg_acc:.2f}')
                t.append(avg_acc)
                if avg_acc >= max_acc:
                    best_C = C
                    best_gamma = gamma
                    max_acc = avg_acc
        gridsearch.append(t)
    gridsearch = np.asarray(gridsearch)
    return gridsearch, (best_C, best_gamma, max_acc)

In [203]:
cand_C = [0.01, 0.1, 1, 10, 100, 1000, 10000]
cand_gamma = [1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1000]
gridsearch, best_parameters = svm_gridsearch(x_train, y_train, kfold_data, cand_C, cand_gamma)
print(f'Best parameter (C, gamma): {best_parameters}')

C=0.01, gamma=0.0001, avg acc=0.69
C=0.01, gamma=0.001, avg acc=0.69
C=0.01, gamma=0.01, avg acc=0.69
C=0.01, gamma=0.1, avg acc=0.69
C=0.01, gamma=1, avg acc=0.69
C=0.01, gamma=10, avg acc=0.69
C=0.01, gamma=100, avg acc=0.69
C=0.01, gamma=1000, avg acc=0.69
C=0.1, gamma=0.0001, avg acc=0.69
C=0.1, gamma=0.001, avg acc=0.69
C=0.1, gamma=0.01, avg acc=0.69
C=0.1, gamma=0.1, avg acc=0.69
C=0.1, gamma=1, avg acc=0.69
C=0.1, gamma=10, avg acc=0.69
C=0.1, gamma=100, avg acc=0.69
C=0.1, gamma=1000, avg acc=0.69
C=1, gamma=0.0001, avg acc=0.69
C=1, gamma=0.001, avg acc=0.69
C=1, gamma=0.01, avg acc=0.69
C=1, gamma=0.1, avg acc=0.69
C=1, gamma=1, avg acc=0.69
C=1, gamma=10, avg acc=0.69
C=1, gamma=100, avg acc=0.69
C=1, gamma=1000, avg acc=0.69
C=10, gamma=0.0001, avg acc=0.76
C=10, gamma=0.001, avg acc=0.89
C=10, gamma=0.01, avg acc=0.69
C=10, gamma=0.1, avg acc=0.69
C=10, gamma=1, avg acc=0.69
C=10, gamma=10, avg acc=0.69
C=10, gamma=100, avg acc=0.69
C=10, gamma=1000, avg acc=0.69
C=100, g

## Question 3
Plot the grid search results of your SVM. The x, y represents the hyperparameters of “gamma” and “C”, respectively. And the color represents the average score of validation folds
You reults should be look like this reference image below ![image](https://miro.medium.com/max/1296/1*wGWTup9r4cVytB5MOnsjdQ.png) 

In [204]:
def annotate_heatmap(im, valfmt, threshold=None):
    data = im.get_array()
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    kw = dict(horizontalalignment="center",
              verticalalignment="center")

    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    texts = []
    textcolors = ['black', 'white']
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)
    return texts

In [205]:
def annotate_heatmap(im, valfmt, threshold=None):
    data = im.get_array()
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    kw = dict(horizontalalignment="center",
              verticalalignment="center")

    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    texts = []
    textcolors = ['black', 'white']
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)
    return texts

In [206]:
def heatmap(data, row_labels, col_labels, ax=None, **kwargs):
    if not ax:
        ax = plt.gca()
    im = ax.imshow(data, **kwargs)
    cbar = ax.figure.colorbar(im, ax=ax)
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    plt.setp(
        ax.get_yticklabels(), rotation=90,
        va='bottom', ha='center', rotation_mode='anchor')
    ax.set_xlabel('Gamma Parameter')
    ax.set_ylabel('C Parameter')
    ax.tick_params(top=False, bottom=True, labeltop=False, labelbottom=True)
    return im, cbar

In [207]:
fig, ax = plt.subplots()
im, cbar = heatmap(gridsearch, cand_C, cand_gamma, ax=ax, cmap='seismic_r')
texts = annotate_heatmap(im, valfmt='{x:.2f}', threshold=0.2)
plt.title('Hyperparameter Gridsearch')
fig.tight_layout()
plt.savefig('gridsearch_svc.png', dpi=300, transparent=True)
plt.clf()

<Figure size 432x288 with 0 Axes>

## Question 4
Train your SVM model by the best parameters you found from question 2 on the whole training set and evaluate the performance on the test set.

In [208]:
kfold_data = cross_validation(x_train, y_train, k=55)
cand_C = [10, 10.5, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
cand_gamma = [1e-3, 1.2*1e-3, 1.4*1e-3, 1.6*1e-3, 1.8*1e-3]
gridsearch, best_parameters = svm_gridsearch(x_train, y_train, kfold_data, cand_C, cand_gamma)
print(f'Best parameter (C, gamma): {best_parameters}')

C=10, gamma=0.001, avg acc=0.90
C=10, gamma=0.0012, avg acc=0.90
C=10, gamma=0.0014, avg acc=0.90
C=10, gamma=0.0016, avg acc=0.90
C=10, gamma=0.0018000000000000002, avg acc=0.89
C=10.5, gamma=0.001, avg acc=0.90
C=10.5, gamma=0.0012, avg acc=0.90
C=10.5, gamma=0.0014, avg acc=0.90
C=10.5, gamma=0.0016, avg acc=0.90
C=10.5, gamma=0.0018000000000000002, avg acc=0.89
C=11, gamma=0.001, avg acc=0.90
C=11, gamma=0.0012, avg acc=0.90
C=11, gamma=0.0014, avg acc=0.90
C=11, gamma=0.0016, avg acc=0.90
C=11, gamma=0.0018000000000000002, avg acc=0.89
C=12, gamma=0.001, avg acc=0.90
C=12, gamma=0.0012, avg acc=0.90
C=12, gamma=0.0014, avg acc=0.90
C=12, gamma=0.0016, avg acc=0.90
C=12, gamma=0.0018000000000000002, avg acc=0.89
C=13, gamma=0.001, avg acc=0.90
C=13, gamma=0.0012, avg acc=0.90
C=13, gamma=0.0014, avg acc=0.90
C=13, gamma=0.0016, avg acc=0.90
C=13, gamma=0.0018000000000000002, avg acc=0.89
C=14, gamma=0.001, avg acc=0.90
C=14, gamma=0.0012, avg acc=0.90
C=14, gamma=0.0014, avg acc=0.

In [209]:
best_C, best_gamma, _ = best_parameters
best_model = SVC(C=best_C, kernel='rbf', gamma=best_gamma)
best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
print("Accuracy score: ", accuracy_score(y_pred, y_test))

Accuracy score:  0.90625
